# MIDAS for GT indexes

In [1]:
pip install midaspy


INFO: pip is looking at multiple versions of midaspy to determine which version is compatible with other requirements. This could take a while.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Cannot install midaspy==1.0.2, midaspy==1.1.0, midaspy==1.1.1, midaspy==1.2.0, midaspy==1.2.1, midaspy==1.2.2 and midaspy==1.2.3 because these package versions have conflicting dependencies.

The conflict is caused by:
    midaspy 1.2.3 depends on tensorflow-addons>=0.11
    midaspy 1.2.2 depends on tensorfl

### Step 1: Import librerie e caricamento tutti i dati

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.special import beta as beta_func
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

print("Setup librerie completato ✓")

# Caricamento tutti i dati necessari
print("Caricamento dati...")

# 1. Dati FRED
compensation = pd.read_csv('/Users/tommaso/Desktop/tesi-inflation-gt/HNKPC/FRED_data/ITACOMPQDSNAQ.csv')
gdp_nominal = pd.read_csv('/Users/tommaso/Desktop/tesi-inflation-gt/HNKPC/FRED_data/ITAGDPNQDSMEI.csv')

# 2. GT monthly destagionalizzati
gt_monthly = pd.read_csv('/Users/tommaso/Desktop/tesi-inflation-gt/Destagionalized_Indexes/dati_preparati_fase1/indici_gt_destag_fase1.csv')

# 3. NIC monthly destagionalizzato
nic_monthly = pd.read_csv('/Users/tommaso/Desktop/tesi-inflation-gt/Destagionalized_Indexes/dati_preparati_fase1/nic_fase1.csv')

print("Tutti i dati caricati:")
print(f"Compensation shape: {compensation.shape}")
print(f"GDP Nominal shape: {gdp_nominal.shape}")
print(f"GT monthly shape: {gt_monthly.shape}")
print(f"NIC monthly shape: {nic_monthly.shape}")

# Mostra strutture per verificare
print("\nStruttura files:")
print("Compensation:", compensation.columns.tolist())
print("GDP:", gdp_nominal.columns.tolist())
print("GT:", gt_monthly.columns.tolist()[:5])  # Prime 5 colonne
print("NIC:", nic_monthly.columns.tolist())


Setup librerie completato ✓
Caricamento dati...
Tutti i dati caricati:
Compensation shape: (81, 2)
GDP Nominal shape: (79, 2)
GT monthly shape: (252, 12)
NIC monthly shape: (252, 2)

Struttura files:
Compensation: ['observation_date', 'ITACOMPQDSNAQ']
GDP: ['observation_date', 'ITAGDPNQDSMEI']
GT: ['Unnamed: 0', 'indice_Inflazione_GT_PCA_SA', 'indice_Tematico_GT_SA', 'indice_Termini_Diretti_Purificato', 'indice_Alimentari_Purificato']
NIC: ['Periodo', 'NIC_destag_ISTAT']


### Step 2: Preparazione dati FRED e calcolo Labor Share

In [3]:
# Prepara dati FRED
compensation.columns = ['DATE', 'COMPENSATION']
gdp_nominal.columns = ['DATE', 'GDP_NOMINAL']

# Converti date
compensation['DATE'] = pd.to_datetime(compensation['DATE'])
gdp_nominal['DATE'] = pd.to_datetime(gdp_nominal['DATE'])

# Definisci periodo comune
start_date = '2004-01-01'
end_date = '2023-09-30'

# Filtra FRED data
compensation_filtered = compensation[
    (compensation['DATE'] >= start_date) & 
    (compensation['DATE'] <= end_date)
].copy()

gdp_filtered = gdp_nominal[
    (gdp_nominal['DATE'] >= start_date) & 
    (gdp_nominal['DATE'] <= end_date)
].copy()

# Merge e calcolo Labor Share
fred_data = pd.merge(compensation_filtered, gdp_filtered, on='DATE', how='inner')
fred_data['LABOR_SHARE'] = (fred_data['COMPENSATION'] / fred_data['GDP_NOMINAL']) * 100

# Crea mapping quarterly
fred_data['YEAR_QUARTER'] = fred_data['DATE'].dt.to_period('Q')

print("Labor Share calcolata:")
print(f"Osservazioni FRED: {fred_data.shape[0]}")
print(f"Periodo: da {fred_data['DATE'].min()} a {fred_data['DATE'].max()}")
print(f"Labor Share - Media: {fred_data['LABOR_SHARE'].mean():.2f}%")
print(f"Labor Share - Range: {fred_data['LABOR_SHARE'].min():.2f}% - {fred_data['LABOR_SHARE'].max():.2f}%")

# Mostra sample
print("\nSample Labor Share:")
print(fred_data[['DATE', 'LABOR_SHARE']].head())


Labor Share calcolata:
Osservazioni FRED: 79
Periodo: da 2004-01-01 00:00:00 a 2023-07-01 00:00:00
Labor Share - Media: 39.46%
Labor Share - Range: 37.40% - 41.21%

Sample Labor Share:
        DATE  LABOR_SHARE
0 2004-01-01    37.395267
1 2004-04-01    37.742574
2 2004-07-01    37.624326
3 2004-10-01    37.526931
4 2005-01-01    38.039639


### Step 3: Preparazione dati GT monthly

In [5]:
# Identifica colonne GT corrette
print("Colonne GT disponibili:")
for i, col in enumerate(gt_monthly.columns):
    print(f"{i}: {col}")

# Seleziona colonne GT (aggiusta i nomi se necessario)
date_col_gt = gt_monthly.columns[0]

# Cerca colonne con inflazione e tematico
gt_cols = []
for col in gt_monthly.columns:
    if 'inflazione' in col.lower() and 'pca' in col.lower():
        gt_cols.append(col)
    elif 'tematico' in col.lower():
        gt_cols.append(col)

print(f"Colonne GT selezionate: {gt_cols}")

# Se non trovate, usa nomi manuali (aggiusta secondo la tua struttura)
if len(gt_cols) < 2:
    print("⚠️ Colonne GT non trovate automaticamente. Specifica manualmente:")
    print("Inserisci i nomi esatti delle colonne per:")
    print("1. Indice Inflazione GT")
    print("2. Indice Tematico GT")
    # Per ora uso nomi tipici - aggiusta se necessario
    gt_cols = ['indice_Inflazione_GT_PCA_SA', 'indice_Tematico_GT_SA']

gt_monthly_clean = gt_monthly[[date_col_gt] + gt_cols].copy()
gt_monthly_clean.columns = ['DATE', 'GT_INFLAZIONE', 'GT_TEMATICO']
gt_monthly_clean['DATE'] = pd.to_datetime(gt_monthly_clean['DATE'])

# Filtra periodo
gt_monthly_clean = gt_monthly_clean[
    (gt_monthly_clean['DATE'] >= start_date) & 
    (gt_monthly_clean['DATE'] <= end_date)
].reset_index(drop=True)

print(f"\nGT monthly preparati: {gt_monthly_clean.shape[0]} osservazioni")
print("Sample GT:")
print(gt_monthly_clean.head())


Colonne GT disponibili:
0: Unnamed: 0
1: indice_Inflazione_GT_PCA_SA
2: indice_Tematico_GT_SA
3: indice_Termini_Diretti_Purificato
4: indice_Alimentari_Purificato
5: indice_Energia_Purificato
6: indice_Abitazione_Purificato
7: indice_Trasporti_Purificato
8: indice_Politiche_Economiche_Purificato
9: indice_Aspettative_Consumatori_Purificato
10: indice_Sanita_Purificato
11: indice_Ricreazione_Purificato
Colonne GT selezionate: ['indice_Inflazione_GT_PCA_SA', 'indice_Tematico_GT_SA']

GT monthly preparati: 237 osservazioni
Sample GT:
        DATE  GT_INFLAZIONE  GT_TEMATICO
0 2004-01-01       3.486825    -6.522004
1 2004-02-01       5.656006    -6.534317
2 2004-03-01       4.820157    -8.017502
3 2004-04-01       4.662026    -6.665120
4 2004-05-01       4.382917    -7.179877


### Step 4: Preparazione dati NIC monthly

In [6]:
# Identifica colonne NIC
print("Colonne NIC disponibili:")
for i, col in enumerate(nic_monthly.columns):
    print(f"{i}: {col}")

date_col_nic = nic_monthly.columns[0]

# Cerca colonna NIC
nic_cols = [col for col in nic_monthly.columns if 'nic' in col.lower()]
print(f"Colonne NIC trovate: {nic_cols}")

if len(nic_cols) == 0:
    print("⚠️ Colonna NIC non trovata. Usa il nome esatto dalla lista sopra")
    # Usa nome tipico - aggiusta se necessario
    nic_col = 'NIC_destag_ISTAT'
else:
    nic_col = nic_cols[0]

nic_monthly_clean = nic_monthly[[date_col_nic, nic_col]].copy()
nic_monthly_clean.columns = ['DATE', 'NIC']
nic_monthly_clean['DATE'] = pd.to_datetime(nic_monthly_clean['DATE'])

# Calcola inflation rate
nic_monthly_clean['INFLATION_RATE'] = nic_monthly_clean['NIC'].pct_change() * 100

# Filtra periodo
nic_monthly_clean = nic_monthly_clean[
    (nic_monthly_clean['DATE'] >= start_date) & 
    (nic_monthly_clean['DATE'] <= end_date)
].reset_index(drop=True)

print(f"\nNIC monthly preparato: {nic_monthly_clean.shape[0]} osservazioni")
print("Sample NIC:")
print(nic_monthly_clean[['DATE', 'NIC', 'INFLATION_RATE']].head())


Colonne NIC disponibili:
0: Periodo
1: NIC_destag_ISTAT
Colonne NIC trovate: ['NIC_destag_ISTAT']

NIC monthly preparato: 237 osservazioni
Sample NIC:
        DATE   NIC  INFLATION_RATE
0 2004-01-01  82.0             NaN
1 2004-02-01  82.2        0.243902
2 2004-03-01  82.5        0.364964
3 2004-04-01  82.7        0.242424
4 2004-05-01  82.9        0.241838


### Step 5: Merge finale e creazione dataset MIDAS

In [ ]:
# Merge dati monthly
monthly_data = pd.merge(nic_monthly_clean[['DATE', 'INFLATION_RATE']], 
                       gt_monthly_clean, on='DATE', how='inner')

# Rimuovi NaN dalla inflation rate
monthly_data = monthly_data.dropna().reset_index(drop=True)

# Crea mapping per Labor Share quarterly
monthly_data['YEAR_QUARTER'] = monthly_data['DATE'].dt.to_period('Q')

# Prepara Labor Share mapping
labor_share_map = fred_data[['YEAR_QUARTER', 'LABOR_SHARE']].copy()

# Merge con Labor Share
midas_data = pd.merge(monthly_data, labor_share_map, on='YEAR_QUARTER', how='inner')

# Aggiungi variabili aggiuntive
midas_data['YEAR'] = midas_data['DATE'].dt.year
midas_data['MONTH'] = midas_data['DATE'].dt.month

print("Dataset MIDAS base creato:")
print(f"Osservazioni: {midas_data.shape[0]}")
print(f"Periodo: da {midas_data['DATE'].min()} a {midas_data['DATE'].max()}")
print(f"Colonne: {midas_data.columns.tolist()}")

print("\nSample dataset finale:")
print(midas_data[['DATE', 'INFLATION_RATE', 'GT_INFLAZIONE', 'GT_TEMATICO', 'LABOR_SHARE']].head())

print("\nStatistiche base:")
print(midas_data[['INFLATION_RATE', 'GT_INFLAZIONE', 'GT_TEMATICO', 'LABOR_SHARE']].describe().round(3))


Dataset MIDAS base creato:
Osservazioni: 236
Periodo: da 2004-02-01 00:00:00 a 2023-09-01 00:00:00
Colonne: ['DATE', 'INFLATION_RATE', 'GT_INFLAZIONE', 'GT_TEMATICO', 'YEAR_QUARTER', 'LABOR_SHARE', 'YEAR', 'MONTH']

Sample dataset finale:
        DATE  INFLATION_RATE  GT_INFLAZIONE  GT_TEMATICO  LABOR_SHARE
0 2004-02-01        0.243902       5.656006    -6.534317    37.395267
1 2004-03-01        0.364964       4.820157    -8.017502    37.395267
2 2004-04-01        0.242424       4.662026    -6.665120    37.742574
3 2004-05-01        0.241838       4.382917    -7.179877    37.742574
4 2004-06-01        0.241255       1.974327    -6.448966    37.742574

Statistiche base:
       INFLATION_RATE  GT_INFLAZIONE  GT_TEMATICO  LABOR_SHARE
count         236.000        236.000      236.000      236.000
mean            0.163         -0.254       -0.363       39.469
std             0.349          2.642        3.904        0.880
min            -0.680         -3.685       -8.018       37.395
25%    

### Step 6: Creazione lag e dummy per MIDAS

In [8]:
# Crea lag per GT basati sui risultati Granger precedenti
midas_data['GT_INFLAZIONE_LAG3'] = midas_data['GT_INFLAZIONE'].shift(3)
midas_data['GT_TEMATICO_LAG1'] = midas_data['GT_TEMATICO'].shift(1)

# Lag inflazione per componente backward-looking HNKPC
midas_data['INFLATION_LAG1'] = midas_data['INFLATION_RATE'].shift(1)

# Dummy outlier (monthly precision)
midas_data['DUMMY_2022_01'] = ((midas_data['DATE'].dt.year == 2022) & 
                               (midas_data['DATE'].dt.month == 1)).astype(int)

midas_data['DUMMY_2022_10'] = ((midas_data['DATE'].dt.year == 2022) & 
                               (midas_data['DATE'].dt.month == 10)).astype(int)

# Rimuovi NaN dai lag
midas_data_clean = midas_data.dropna().reset_index(drop=True)

print("Variabili MIDAS preparate:")
print(f"Osservazioni finali: {midas_data_clean.shape[0]}")
print(f"Periodo: da {midas_data_clean['DATE'].min()} a {midas_data_clean['DATE'].max()}")

# Verifica outlier
outlier_2022_01 = midas_data_clean[midas_data_clean['DUMMY_2022_01'] == 1]
outlier_2022_10 = midas_data_clean[midas_data_clean['DUMMY_2022_10'] == 1]

print(f"\nOutlier gennaio 2022: {len(outlier_2022_01)} osservazioni")
if len(outlier_2022_01) > 0:
    print(f"Inflazione gennaio 2022: {outlier_2022_01['INFLATION_RATE'].values[0]:.2f}%")

print(f"Outlier ottobre 2022: {len(outlier_2022_10)} osservazioni")
if len(outlier_2022_10) > 0:
    print(f"Inflazione ottobre 2022: {outlier_2022_10['INFLATION_RATE'].values[0]:.2f}%")

# Mostra statistiche finali
key_vars = ['INFLATION_RATE', 'LABOR_SHARE', 'GT_INFLAZIONE_LAG3', 'GT_TEMATICO_LAG1', 'INFLATION_LAG1']
print("\nStatistiche variabili chiave:")
print(midas_data_clean[key_vars].describe().round(3))


Variabili MIDAS preparate:
Osservazioni finali: 233
Periodo: da 2004-05-01 00:00:00 a 2023-09-01 00:00:00

Outlier gennaio 2022: 1 osservazioni
Inflazione gennaio 2022: 1.59%
Outlier ottobre 2022: 1 osservazioni
Inflazione ottobre 2022: 3.42%

Statistiche variabili chiave:
       INFLATION_RATE  LABOR_SHARE  GT_INFLAZIONE_LAG3  GT_TEMATICO_LAG1  \
count         233.000      233.000             233.000           233.000   
mean            0.162       39.494              -0.346            -0.336   
std             0.351        0.856               2.529             3.843   
min            -0.680       37.527              -3.685            -7.180   
25%             0.000       39.216              -2.057            -3.568   
50%             0.118       39.497              -1.351            -0.139   
75%             0.313       40.158               0.640             2.427   
max             3.415       41.206               8.886            11.504   

       INFLATION_LAG1  
count         233

### Step 7: Implementazione classe MIDAS custom

In [9]:
class MIDAS_Regressor:
    """
    Implementazione MIDAS semplificata per mixed-frequency data
    """
    
    def __init__(self, m=3):
        """
        m: numero di osservazioni high-freq per low-freq (3 mesi per trimestre)
        """
        self.m = m
        self.params = None
        self.fitted_values = None
        self.residuals = None
        self.aic = None
        self.bic = None
        self.rsquared = None
        
    def beta_weights(self, theta1, theta2):
        """Calcola pesi MIDAS usando distribuzione Beta"""
        k = np.arange(1, self.m + 1)
        weights = (k / self.m)**(theta1 - 1) * (1 - k / self.m)**(theta2 - 1)
        # Normalizza
        weights = weights / np.sum(weights)
        return weights
    
    def create_midas_matrix(self, y, x_high_freq, x_low_freq, theta1, theta2):
        """Crea matrice X per regressione MIDAS"""
        n_obs = len(y)
        weights = self.beta_weights(theta1, theta2)
        
        # Inizializza matrice X con costante
        X = np.ones((n_obs, 1))
        
        # Aggiungi variabili low-frequency (Labor Share)
        if x_low_freq is not None:
            if len(x_low_freq.shape) == 1:
                X = np.column_stack([X, x_low_freq])
            else:
                X = np.column_stack([X, x_low_freq])
        
        # Aggiungi variabili high-frequency pesate con MIDAS
        for col_idx in range(x_high_freq.shape[1]):
            weighted_series = np.zeros(n_obs)
            
            for obs_idx in range(self.m, n_obs):  # Inizia da m per avere abbastanza lag
                # Prendi m osservazioni precedenti
                lag_values = x_high_freq.iloc[obs_idx-self.m:obs_idx, col_idx].values
                # Applica pesi MIDAS (inverti ordine per lag structure)
                weighted_series[obs_idx] = np.sum(lag_values[::-1] * weights)
            
            X = np.column_stack([X, weighted_series])
        
        return X
    
    def objective_function(self, params, y, x_high_freq, x_low_freq):
        """Funzione obiettivo da minimizzare (SSR)"""
        try:
            # Parametri: [theta1, theta2, betas...]
            theta1, theta2 = params[:2]
            betas = params[2:]
            
            # Crea matrice X
            X = self.create_midas_matrix(y, x_high_freq, x_low_freq, theta1, theta2)
            
            # Rimuovi prime m osservazioni (per lag structure)
            X_clean = X[self.m:]
            y_clean = y[self.m:]
            
            # Regressione OLS
            fitted_values = X_clean @ betas
            residuals = y_clean - fitted_values
            
            return np.sum(residuals**2)
        
        except:
            return 1e10  # Penalizza parametri non validi
    
    def fit(self, y, x_high_freq, x_low_freq=None, initial_theta=[1.5, 1.5]):
        """Stima modello MIDAS"""
        
        # Conta parametri
        n_high = x_high_freq.shape[1]
        n_low = 1 if x_low_freq is not None else 0
        n_betas = 1 + n_low + n_high  # costante + low-freq + high-freq
        
        # Parametri iniziali: [theta1, theta2, beta_const, beta_low, beta_high...]
        initial_params = initial_theta + [0.1] * n_betas
        
        # Bounds: theta > 0.1, betas liberi
        bounds = [(0.1, 10), (0.1, 10)] + [(-10, 10)] * n_betas
        
        # Ottimizzazione
        result = minimize(
            self.objective_function,
            initial_params,
            args=(y, x_high_freq, x_low_freq),
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 1000}
        )
        
        if result.success:
            self.params = result.x
            
            # Calcola fitted values e residui
            theta1, theta2 = self.params[:2]
            betas = self.params[2:]
            
            X = self.create_midas_matrix(y, x_high_freq, x_low_freq, theta1, theta2)
            X_clean = X[self.m:]
            y_clean = y[self.m:]
            
            self.fitted_values = X_clean @ betas
            self.residuals = y_clean - self.fitted_values
            
            # Calcola metriche
            n_obs = len(y_clean)
            k_params = len(betas)
            
            ssr = np.sum(self.residuals**2)
            tss = np.sum((y_clean - np.mean(y_clean))**2)
            
            self.rsquared = 1 - ssr/tss
            self.aic = n_obs * np.log(ssr/n_obs) + 2 * k_params
            self.bic = n_obs * np.log(ssr/n_obs) + k_params * np.log(n_obs)
            
            return True
        else:
            print(f"Ottimizzazione fallita: {result.message}")
            return False

print("Classe MIDAS implementata ✓")


Classe MIDAS implementata ✓


### Step 8: Test della classe MIDAS

In [10]:
# Test su dataset semplificato
print("Test classe MIDAS...")

# Prepara variabili per test
y_test = midas_data_clean['INFLATION_RATE'].values
x_high_test = midas_data_clean[['GT_INFLAZIONE_LAG3', 'GT_TEMATICO_LAG1']]
x_low_test = midas_data_clean['LABOR_SHARE'].values

print(f"Variabili test:")
print(f"y (inflation): {len(y_test)} obs, range: {y_test.min():.2f} to {y_test.max():.2f}")
print(f"x_high (GT): {x_high_test.shape}")
print(f"x_low (labor_share): {len(x_low_test)} obs, range: {x_low_test.min():.1f} to {x_low_test.max():.1f}")

# Istanzia e stima MIDAS
midas = MIDAS_Regressor(m=3)

print("\nStima MIDAS in corso...")
success = midas.fit(y_test, x_high_test, x_low_test)

if success:
    print("✅ MIDAS stimato con successo!")
    print(f"Parametri: {midas.params}")
    print(f"R-squared: {midas.rsquared:.4f}")
    print(f"AIC: {midas.aic:.2f}")
    print(f"BIC: {midas.bic:.2f}")
    
    # Mostra pesi MIDAS stimati
    theta1, theta2 = midas.params[:2]
    weights = midas.beta_weights(theta1, theta2)
    print(f"\nPesi MIDAS stimati:")
    for i, w in enumerate(weights):
        print(f"  Lag {i+1}: {w:.3f}")
else:
    print("❌ Stima MIDAS fallita")


Test classe MIDAS...
Variabili test:
y (inflation): 233 obs, range: -0.68 to 3.42
x_high (GT): (233, 2)
x_low (labor_share): 233 obs, range: 37.5 to 41.2

Stima MIDAS in corso...
Ottimizzazione fallita: ABNORMAL: 
❌ Stima MIDAS fallita


### Step 9: Versione MIDAS robusta con debug

In [11]:
class MIDAS_Regressor_Robust:
    """
    Versione robusta di MIDAS con debug e gestione errori migliorata
    """
    
    def __init__(self, m=3):
        self.m = m
        self.params = None
        self.fitted_values = None
        self.residuals = None
        self.aic = None
        self.bic = None
        self.rsquared = None
        self.debug_info = {}
        
    def beta_weights(self, theta1, theta2):
        """Calcola pesi MIDAS con controlli numerici"""
        try:
            # Assicura che theta siano positivi
            theta1 = max(theta1, 0.1)
            theta2 = max(theta2, 0.1)
            
            k = np.arange(1, self.m + 1) / self.m  # Normalizza a [0,1]
            
            # Calcola pesi Beta
            weights = k**(theta1 - 1) * (1 - k)**(theta2 - 1)
            
            # Controlla per NaN o infiniti
            if np.any(~np.isfinite(weights)):
                return np.ones(self.m) / self.m  # Pesi uniformi come fallback
            
            # Normalizza
            weights = weights / np.sum(weights)
            return weights
            
        except:
            return np.ones(self.m) / self.m  # Fallback sicuro
    
    def create_simple_midas(self, y, x_high_freq, x_low_freq, theta1, theta2):
        """Versione semplificata per debug"""
        n_obs = len(y)
        weights = self.beta_weights(theta1, theta2)
        
        # Matrice X: [costante, labor_share, GT_weighted1, GT_weighted2]
        X = np.ones((n_obs, 1))  # Costante
        
        # Labor Share (low frequency)
        X = np.column_stack([X, x_low_freq])
        
        # GT pesati (high frequency) - versione semplificata
        for col_idx in range(x_high_freq.shape[1]):
            weighted_col = np.zeros(n_obs)
            
            # Per ogni osservazione, calcola media pesata degli ultimi m valori
            for i in range(self.m, n_obs):
                lag_values = x_high_freq.iloc[i-self.m:i, col_idx].values
                weighted_col[i] = np.sum(lag_values * weights[::-1])  # Inverti per struttura lag
            
            X = np.column_stack([X, weighted_col])
        
        return X
    
    def objective_simple(self, params, y, x_high_freq, x_low_freq):
        """Funzione obiettivo semplificata"""
        try:
            theta1, theta2 = params[:2]
            betas = params[2:]
            
            # Crea matrice X
            X = self.create_simple_midas(y, x_high_freq, x_low_freq, theta1, theta2)
            
            # Usa solo osservazioni con dati completi
            valid_idx = self.m
            X_valid = X[valid_idx:]
            y_valid = y[valid_idx:]
            
            # OLS
            y_pred = X_valid @ betas
            residuals = y_valid - y_pred
            ssr = np.sum(residuals**2)
            
            # Penalizza se SSR non è finito
            if not np.isfinite(ssr):
                return 1e10
                
            return ssr
            
        except Exception as e:
            self.debug_info['last_error'] = str(e)
            return 1e10
    
    def fit_simple(self, y, x_high_freq, x_low_freq):
        """Stima semplificata con multiple starting points"""
        
        # Numero parametri: 2 theta + 4 beta (const, labor_share, GT1, GT2)
        n_params = 2 + 4
        
        # Multiple starting points per robustezza
        starting_points = [
            [1.0, 1.0, 0.0, 0.1, 0.05, 0.05],  # Conservative
            [2.0, 2.0, 0.0, 0.2, 0.1, 0.1],    # Moderate
            [0.5, 0.5, 0.0, 0.05, 0.02, 0.02], # Aggressive
        ]
        
        best_result = None
        best_objective = np.inf
        
        for i, start_params in enumerate(starting_points):
            print(f"Tentativo {i+1}/3 con starting point: {start_params[:2]}")
            
            bounds = [(0.1, 5.0), (0.1, 5.0)] + [(-1.0, 1.0)] * 4
            
            try:
                result = minimize(
                    self.objective_simple,
                    start_params,
                    args=(y, x_high_freq, x_low_freq),
                    method='L-BFGS-B',
                    bounds=bounds,
                    options={'maxiter': 500, 'ftol': 1e-6}
                )
                
                if result.fun < best_objective:
                    best_result = result
                    best_objective = result.fun
                    
                print(f"  Risultato: success={result.success}, objective={result.fun:.2f}")
                
            except Exception as e:
                print(f"  Errore: {e}")
                continue
        
        if best_result is not None and best_result.success:
            self.params = best_result.x
            
            # Calcola statistiche finali
            theta1, theta2 = self.params[:2]
            betas = self.params[2:]
            
            X = self.create_simple_midas(y, x_high_freq, x_low_freq, theta1, theta2)
            X_valid = X[self.m:]
            y_valid = y[self.m:]
            
            self.fitted_values = X_valid @ betas
            self.residuals = y_valid - self.fitted_values
            
            # Metriche
            n_obs = len(y_valid)
            ssr = np.sum(self.residuals**2)
            tss = np.sum((y_valid - np.mean(y_valid))**2)
            
            self.rsquared = 1 - ssr/tss
            self.aic = n_obs * np.log(ssr/n_obs) + 2 * len(betas)
            self.bic = n_obs * np.log(ssr/n_obs) + len(betas) * np.log(n_obs)
            
            return True
        else:
            print("Tutti i tentativi di ottimizzazione sono falliti")
            return False

print("Classe MIDAS robusta implementata ✓")


Classe MIDAS robusta implementata ✓


### Step 10: Test della versione robusta

In [12]:
# Test della versione robusta
print("Test MIDAS robusta...")

# Verifica dati input
print(f"Controllo dati input:")
print(f"y NaN: {np.sum(np.isnan(y_test))}")
print(f"x_high NaN: {np.sum(np.isnan(x_high_test.values))}")
print(f"x_low NaN: {np.sum(np.isnan(x_low_test))}")

# Istanzia versione robusta
midas_robust = MIDAS_Regressor_Robust(m=3)

print("\nStima MIDAS robusta...")
success = midas_robust.fit_simple(y_test, x_high_test, x_low_test)

if success:
    print("\n✅ MIDAS robusta stimata con successo!")
    
    # Risultati
    theta1, theta2 = midas_robust.params[:2]
    betas = midas_robust.params[2:]
    
    print(f"\nParametri MIDAS:")
    print(f"  Theta1 (primo param Beta): {theta1:.3f}")
    print(f"  Theta2 (secondo param Beta): {theta2:.3f}")
    print(f"  Beta costante: {betas[0]:.3f}")
    print(f"  Beta Labor Share: {betas[1]:.3f}")
    print(f"  Beta GT Inflazione: {betas[2]:.3f}")
    print(f"  Beta GT Tematico: {betas[3]:.3f}")
    
    print(f"\nMetriche:")
    print(f"  R-squared: {midas_robust.rsquared:.4f}")
    print(f"  AIC: {midas_robust.aic:.2f}")
    print(f"  BIC: {midas_robust.bic:.2f}")
    print(f"  RMSE: {np.sqrt(np.mean(midas_robust.residuals**2)):.3f}")
    
    # Pesi MIDAS
    weights = midas_robust.beta_weights(theta1, theta2)
    print(f"\nPesi MIDAS:")
    for i, w in enumerate(weights):
        print(f"  Mese t-{3-i}: {w:.3f}")
        
else:
    print("❌ Anche la versione robusta è fallita")
    if 'last_error' in midas_robust.debug_info:
        print(f"Ultimo errore: {midas_robust.debug_info['last_error']}")


Test MIDAS robusta...
Controllo dati input:
y NaN: 0
x_high NaN: 0
x_low NaN: 0

Stima MIDAS robusta...
Tentativo 1/3 con starting point: [1.0, 1.0]
  Risultato: success=False, objective=3272.16
Tentativo 2/3 con starting point: [2.0, 2.0]
  Risultato: success=True, objective=26.27
Tentativo 3/3 con starting point: [0.5, 0.5]
  Risultato: success=True, objective=26.85

✅ MIDAS robusta stimata con successo!

Parametri MIDAS:
  Theta1 (primo param Beta): 0.100
  Theta2 (secondo param Beta): 3.796
  Beta costante: 0.496
  Beta Labor Share: -0.008
  Beta GT Inflazione: 0.037
  Beta GT Tematico: 0.012

Metriche:
  R-squared: 0.0791
  AIC: -490.99
  BIC: -477.24
  RMSE: 0.338

Pesi MIDAS:
  Mese t-3: 0.928
  Mese t-2: 0.072
  Mese t-1: 0.000


### Step 11: Stima modelli di confronto per valutazione MIDAS

In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Prepara dataset per confronti (stesso sample MIDAS)
# MIDAS usa osservazioni da index m=3 in poi
start_idx = 3
comparison_data = midas_data_clean.iloc[start_idx:].copy().reset_index(drop=True)

print(f"Dataset confronto: {len(comparison_data)} osservazioni")
print(f"Periodo: da {comparison_data['DATE'].min()} a {comparison_data['DATE'].max()}")

# Variabile dipendente
y_comp = comparison_data['INFLATION_RATE'].values

print(f"Variabile dipendente - Range: {y_comp.min():.2f} to {y_comp.max():.2f}")


Dataset confronto: 230 osservazioni
Periodo: da 2004-08-01 00:00:00 a 2023-09-01 00:00:00
Variabile dipendente - Range: -0.68 to 3.42


### Step 12: Modello 1 - HNKPC semplice (senza GT)

In [14]:
# HNKPC base: π_t = const + λ*LS_t + γ_b*π_{t-1} + dummy + ε_t
X_hnkpc_base = comparison_data[['LABOR_SHARE', 'INFLATION_LAG1', 'DUMMY_2022_01', 'DUMMY_2022_10']].copy()
X_hnkpc_base = sm.add_constant(X_hnkpc_base)

# Stima OLS con errori robusti
model_hnkpc_base = sm.OLS(y_comp, X_hnkpc_base).fit(cov_type='HAC', cov_kwds={'maxlags': 3})

print("=== MODELLO 1: HNKPC BASE ===")
print(f"R-squared: {model_hnkpc_base.rsquared:.4f}")
print(f"AIC: {model_hnkpc_base.aic:.2f}")
print(f"BIC: {model_hnkpc_base.bic:.2f}")

# Calcola RMSE manualmente
fitted_hnkpc_base = model_hnkpc_base.fittedvalues
rmse_hnkpc_base = np.sqrt(mean_squared_error(y_comp, fitted_hnkpc_base))
print(f"RMSE: {rmse_hnkpc_base:.3f}")

# Coefficienti significativi
print("\nCoefficienti:")
for i, (name, coef, pval) in enumerate(zip(X_hnkpc_base.columns, model_hnkpc_base.params, model_hnkpc_base.pvalues)):
    sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
    print(f"  {name}: {coef:.4f} {sig} (p={pval:.3f})")


=== MODELLO 1: HNKPC BASE ===
R-squared: 0.4779
AIC: 33.21
BIC: 50.40
RMSE: 0.254

Coefficienti:
  const: 0.2094  (p=0.796)
  LABOR_SHARE: -0.0024  (p=0.906)
  INFLATION_LAG1: 0.1779 ** (p=0.001)
  DUMMY_2022_01: 1.4172 *** (p=0.000)
  DUMMY_2022_10: 3.2580 *** (p=0.000)


### Step 13: Modello 2 - HNKPC + GT (OLS tradizionale)

In [15]:
# HNKPC + GT: π_t = const + λ*LS_t + γ_b*π_{t-1} + α1*GT_Infl + α2*GT_Tem + dummy + ε_t
X_hnkpc_gt = comparison_data[['LABOR_SHARE', 'INFLATION_LAG1', 'GT_INFLAZIONE_LAG3', 'GT_TEMATICO_LAG1', 'DUMMY_2022_01', 'DUMMY_2022_10']].copy()
X_hnkpc_gt = sm.add_constant(X_hnkpc_gt)

# Stima OLS
model_hnkpc_gt = sm.OLS(y_comp, X_hnkpc_gt).fit(cov_type='HAC', cov_kwds={'maxlags': 3})

print("\n=== MODELLO 2: HNKPC + GT (OLS) ===")
print(f"R-squared: {model_hnkpc_gt.rsquared:.4f}")
print(f"AIC: {model_hnkpc_gt.aic:.2f}")
print(f"BIC: {model_hnkpc_gt.bic:.2f}")

fitted_hnkpc_gt = model_hnkpc_gt.fittedvalues
rmse_hnkpc_gt = np.sqrt(mean_squared_error(y_comp, fitted_hnkpc_gt))
print(f"RMSE: {rmse_hnkpc_gt:.3f}")

print("\nCoefficienti:")
for i, (name, coef, pval) in enumerate(zip(X_hnkpc_gt.columns, model_hnkpc_gt.params, model_hnkpc_gt.pvalues)):
    sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
    print(f"  {name}: {coef:.4f} {sig} (p={pval:.3f})")



=== MODELLO 2: HNKPC + GT (OLS) ===
R-squared: 0.4930
AIC: 30.46
BIC: 54.53
RMSE: 0.251

Coefficienti:
  const: -0.4266  (p=0.689)
  LABOR_SHARE: 0.0140  (p=0.605)
  INFLATION_LAG1: 0.1417 ** (p=0.008)
  GT_INFLAZIONE_LAG3: 0.0192 * (p=0.043)
  GT_TEMATICO_LAG1: -0.0031  (p=0.630)
  DUMMY_2022_01: 1.3997 *** (p=0.000)
  DUMMY_2022_10: 3.1010 *** (p=0.000)


### Step 14: Confronto finale dei tre modelli

In [16]:
# Raccogli risultati
results_summary = pd.DataFrame({
    'Modello': ['HNKPC_Base', 'HNKPC_GT_OLS', 'MIDAS_GT'],
    'R_squared': [
        model_hnkpc_base.rsquared,
        model_hnkpc_gt.rsquared, 
        midas_robust.rsquared
    ],
    'AIC': [
        model_hnkpc_base.aic,
        model_hnkpc_gt.aic,
        midas_robust.aic
    ],
    'BIC': [
        model_hnkpc_base.bic,
        model_hnkpc_gt.bic,
        midas_robust.bic
    ],
    'RMSE': [
        rmse_hnkpc_base,
        rmse_hnkpc_gt,
        np.sqrt(np.mean(midas_robust.residuals**2))
    ]
})

print("=== CONFRONTO FINALE MODELLI ===")
print(results_summary.round(4))

# Test di significatività miglioramento
def calculate_improvement(base_rmse, model_rmse):
    return ((base_rmse - model_rmse) / base_rmse) * 100

print(f"\nMiglioramenti RMSE rispetto a HNKPC Base:")
print(f"HNKPC + GT (OLS): {calculate_improvement(rmse_hnkpc_base, rmse_hnkpc_gt):+.2f}%")
print(f"MIDAS + GT: {calculate_improvement(rmse_hnkpc_base, np.sqrt(np.mean(midas_robust.residuals**2))):+.2f}%")

# Evidenzia il migliore per ogni metrica
print(f"\nMigliore per metrica:")
print(f"R-squared: {results_summary.loc[results_summary['R_squared'].idxmax(), 'Modello']}")
print(f"AIC (più basso): {results_summary.loc[results_summary['AIC'].idxmin(), 'Modello']}")
print(f"BIC (più basso): {results_summary.loc[results_summary['BIC'].idxmin(), 'Modello']}")
print(f"RMSE (più basso): {results_summary.loc[results_summary['RMSE'].idxmin(), 'Modello']}")


=== CONFRONTO FINALE MODELLI ===
        Modello  R_squared       AIC       BIC    RMSE
0    HNKPC_Base     0.4779   33.2144   50.4048  0.2545
1  HNKPC_GT_OLS     0.4930   30.4593   54.5258  0.2508
2      MIDAS_GT     0.0791 -490.9905 -477.2382  0.3380

Miglioramenti RMSE rispetto a HNKPC Base:
HNKPC + GT (OLS): +1.46%
MIDAS + GT: -32.80%

Migliore per metrica:
R-squared: HNKPC_GT_OLS
AIC (più basso): MIDAS_GT
BIC (più basso): MIDAS_GT
RMSE (più basso): HNKPC_GT_OLS


### Step 15: Analisi dei pesi MIDAS e interpretazione

In [17]:
# Analisi dettagliata pesi MIDAS
theta1, theta2 = midas_robust.params[:2]
weights = midas_robust.beta_weights(theta1, theta2)

print("=== ANALISI PESI MIDAS ===")
print(f"Parametri Beta distribution:")
print(f"  Theta1: {theta1:.3f}")
print(f"  Theta2: {theta2:.3f}")

print(f"\nStruttura temporale pesi:")
total_weight = 0
for i in range(3):
    lag_months = 3 - i
    print(f"  Mese t-{lag_months}: {weights[i]:.1%}")
    total_weight += weights[i]
print(f"  Totale: {total_weight:.1%}")

# Interpretazione economica
print(f"\nInterpretazione:")
if weights[0] > 0.8:
    print("- FORTE concentrazione su lag t-3: GT anticipano inflazione di 3 mesi")
elif weights[0] > 0.5:
    print("- MODERATA concentrazione su lag t-3: effetto ritardato prevalente")
else:
    print("- Distribuzione UNIFORME: effetti distribuiti nel tempo")

# Significatività coefficienti GT in MIDAS
gt_infl_coef = midas_robust.params[4]  # Beta GT Inflazione
gt_tem_coef = midas_robust.params[5]   # Beta GT Tematico

print(f"\nCoefficienti GT in MIDAS:")
print(f"  GT Inflazione: {gt_infl_coef:.4f}")
print(f"  GT Tematico: {gt_tem_coef:.4f}")

if gt_infl_coef > gt_tem_coef:
    print("- GT Inflazione ha impatto MAGGIORE di GT Tematico")
else:
    print("- GT Tematico ha impatto MAGGIORE di GT Inflazione")


=== ANALISI PESI MIDAS ===
Parametri Beta distribution:
  Theta1: 0.100
  Theta2: 3.796

Struttura temporale pesi:
  Mese t-3: 92.8%
  Mese t-2: 7.2%
  Mese t-1: 0.0%
  Totale: 100.0%

Interpretazione:
- FORTE concentrazione su lag t-3: GT anticipano inflazione di 3 mesi

Coefficienti GT in MIDAS:
  GT Inflazione: 0.0366
  GT Tematico: 0.0117
- GT Inflazione ha impatto MAGGIORE di GT Tematico


I risultati sono contrastanti e rivelano problemi metodologici.

Problema principale: i risultati MIDAS mostrano anomalie evidenti:

- AIC MIDAS = -490: Impossibilmente basso vs. HNKPC ~30-50
- R² MIDAS = 0.08: Molto inferiore a HNKPC ~0.48-0.49
- RMSE MIDAS peggiore: 0.338 vs. 0.25x degli altri modelli

Diagnosi del problema
L'AIC negativo indica:

- Possibile errore nel calcolo (log di un numero molto piccolo)
- Sample size differente tra modelli
- Scaling problem nella stima MIDAS

### Step 16: Debug e correzione MIDAS

In [18]:
# Debug dettagliato MIDAS
print("=== DEBUG MIDAS ===")

# Verifica sample sizes
print(f"Sample sizes:")
print(f"MIDAS: {len(midas_robust.residuals)} osservazioni")
print(f"HNKPC: {len(comparison_data)} osservazioni")
print(f"Y originale: {len(y_test)} osservazioni")

# Verifica residui MIDAS
residuals = midas_robust.residuals
print(f"\nResidui MIDAS:")
print(f"Media: {np.mean(residuals):.6f}")
print(f"Std: {np.std(residuals):.6f}")
print(f"Min: {np.min(residuals):.3f}")
print(f"Max: {np.max(residuals):.3f}")
print(f"SSR: {np.sum(residuals**2):.6f}")

# Ricalcola AIC manualmente
n_obs = len(residuals)
k_params = 6  # 2 theta + 4 beta
ssr = np.sum(residuals**2)
mse = ssr / n_obs

print(f"\nRicalcolo AIC:")
print(f"N osservazioni: {n_obs}")
print(f"K parametri: {k_params}")
print(f"SSR: {ssr:.6f}")
print(f"MSE: {mse:.6f}")
print(f"Log(MSE): {np.log(mse):.6f}")

# AIC corretto
aic_manual = n_obs * np.log(mse) + 2 * k_params
print(f"AIC manuale: {aic_manual:.2f}")

# Confronta con MIDAS calcolato
print(f"AIC MIDAS class: {midas_robust.aic:.2f}")
print(f"Differenza: {aic_manual - midas_robust.aic:.2f}")


=== DEBUG MIDAS ===
Sample sizes:
MIDAS: 230 osservazioni
HNKPC: 230 osservazioni
Y originale: 233 osservazioni

Residui MIDAS:
Media: -0.000247
Std: 0.337982
Min: -0.985
Max: 2.877
SSR: 26.273358

Ricalcolo AIC:
N osservazioni: 230
K parametri: 6
SSR: 26.273358
MSE: 0.114232
Log(MSE): -2.169524
AIC manuale: -486.99
AIC MIDAS class: -490.99
Differenza: 4.00


### Step 17: Implementazione MIDAS corretta

In [19]:
# Implementa MIDAS con calcolo metriche corretto
class MIDAS_Fixed:
    def __init__(self, midas_object, y_original, start_idx=3):
        self.midas = midas_object
        self.y_original = y_original
        self.start_idx = start_idx
        
    def recalculate_metrics(self):
        """Ricalcola metriche con metodologia consistente"""
        residuals = self.midas.residuals
        y_subset = self.y_original[self.start_idx:]
        
        n_obs = len(residuals)
        k_params = len(self.midas.params)
        
        # SSR e MSE
        ssr = np.sum(residuals**2)
        mse = ssr / n_obs
        
        # TSS per R-squared
        y_mean = np.mean(y_subset)
        tss = np.sum((y_subset - y_mean)**2)
        
        # Metriche corrette
        self.rsquared_corrected = 1 - ssr/tss
        self.aic_corrected = n_obs * np.log(mse) + 2 * k_params
        self.bic_corrected = n_obs * np.log(mse) + k_params * np.log(n_obs)
        self.rmse_corrected = np.sqrt(mse)
        
        return {
            'rsquared': self.rsquared_corrected,
            'aic': self.aic_corrected, 
            'bic': self.bic_corrected,
            'rmse': self.rmse_corrected,
            'n_obs': n_obs,
            'k_params': k_params
        }

# Applica correzione
midas_fixed = MIDAS_Fixed(midas_robust, y_test, start_idx=3)
metrics_corrected = midas_fixed.recalculate_metrics()

print("\n=== MIDAS CORRETTO ===")
for key, value in metrics_corrected.items():
    if key in ['n_obs', 'k_params']:
        print(f"{key}: {value}")
    else:
        print(f"{key}: {value:.4f}")



=== MIDAS CORRETTO ===
rsquared: 0.0791
aic: -486.9905
bic: -466.3620
rmse: 0.3380
n_obs: 230
k_params: 6


### Step 18: Confronto finale corretto

In [ ]:
# Confronto con metriche corrette
results_corrected = pd.DataFrame({
    'Modello': ['HNKPC_Base', 'HNKPC_GT_OLS', 'MIDAS_GT_Corrected'],
    'R_squared': [
        model_hnkpc_base.rsquared,
        model_hnkpc_gt.rsquared, 
        metrics_corrected['rsquared']
    ],
    'AIC': [
        model_hnkpc_base.aic,
        model_hnkpc_gt.aic,
        metrics_corrected['aic']
    ],
    'BIC': [
        model_hnkpc_base.bic,
        model_hnkpc_gt.bic,
        metrics_corrected['bic']
    ],
    'RMSE': [
        rmse_hnkpc_base,
        rmse_hnkpc_gt,
        metrics_corrected['rmse']
    ],
    'N_obs': [
        len(y_comp),
        len(y_comp),
        metrics_corrected['n_obs']
    ]
})

print("=== CONFRONTO CORRETTO ===")
print(results_corrected.round(4))

# Ricalcola miglioramenti
base_rmse = rmse_hnkpc_base
ols_rmse = rmse_hnkpc_gt  
midas_rmse = metrics_corrected['rmse']

print(f"\nMiglioramenti RMSE vs HNKPC Base:")
print(f"HNKPC + GT (OLS): {((base_rmse - ols_rmse) / base_rmse) * 100:+.2f}%")
print(f"MIDAS + GT: {((base_rmse - midas_rmse) / base_rmse) * 100:+.2f}%")

# Valutazione finale
print(f"\n=== VALUTAZIONE FINALE ===")
if metrics_corrected['aic'] < model_hnkpc_gt.aic:
    print("✅ MIDAS superiore ad HNKPC+GT per AIC")
else:
    print("❌ MIDAS non superiore ad HNKPC+GT per AIC")

if metrics_corrected['rmse'] < rmse_hnkpc_gt:
    print("✅ MIDAS superiore ad HNKPC+GT per RMSE")
else:
    print("❌ MIDAS non superiore ad HNKPC+GT per RMSE")


=== CONFRONTO CORRETTO ===
              Modello  R_squared       AIC       BIC    RMSE  N_obs
0          HNKPC_Base     0.4779   33.2144   50.4048  0.2545    230
1        HNKPC_GT_OLS     0.4930   30.4593   54.5258  0.2508    230
2  MIDAS_GT_Corrected     0.0791 -486.9905 -466.3620  0.3380    230

Miglioramenti RMSE vs HNKPC Base:
HNKPC + GT (OLS): +1.46%
MIDAS + GT: -32.80%

=== VALUTAZIONE FINALE ===
✅ MIDAS superiore ad HNKPC+GT per AIC
❌ MIDAS non superiore ad HNKPC+GT per RMSE
